In [ ]:
# ! pip install nbformat>=4.2.0

In [2]:
import os
import sys
from time import sleep
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

absolute_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(absolute_parent)

from tdf_utility.trading.nse_api import NSE_API, get_nse_india_vix, get_nse_market_status_daily
from tdf_utility.trading.ep_api import fetch_fii_dii_data

nse_api = NSE_API()

def load_graph_data_to_df(object: dict):
    identifier = object.get('data').get('identifier')
    # name = objecct.get('data').get('name')
    grapth_data = object.get('data').get('grapthData')
    df = pd.DataFrame(grapth_data, columns=['Timestamp', 'Price', 'Code'])
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    return df, identifier


def sourcing_data():
        duration = "1Y"
        year = "2025"
        from_dt = "31-12-2024"
        to_dt = "30-12-2025"

        nse_api = NSE_API()
        
        nifty_50_historical_data = nse_api._get_data(f"api/NextApi/apiClient/historicalGraph?functionName=getIndexChart&&index=NIFTY%2050&flag={duration}")

        india_vix_historical_data = get_nse_india_vix()
        india_vix_historical_data['date'] = pd.to_datetime(india_vix_historical_data['date'], format='%d-%b-%Y')

        sleep(3)
        gold_historical_data = nse_api._get_data(f"api/historical-spot-price?symbol=GOLD&fromDate={from_dt}&toDate={to_dt}")
        gold_historical_df = pd.DataFrame(gold_historical_data['data']) \
                                            .loc[:, ['UpdatedDate', 'SpotPrice2']] \
                                            .rename(columns={'UpdatedDate': 'Date', 'SpotPrice2': 'Gold 10gm'})
        gold_historical_df['Date'] = pd.to_datetime(gold_historical_df['Date'], format='%d-%b-%Y')
        gold_historical_df['Gold 10gm'] = gold_historical_df['Gold 10gm'].astype(float)

        sleep(3)
        silver_historical_data = nse_api._get_data(f"api/historical-spot-price?symbol=SILVER&fromDate={from_dt}&toDate={to_dt}")
        silver_historical_df = pd.DataFrame(silver_historical_data['data']) \
                                            .loc[:, ['UpdatedDate', 'SpotPrice2']] \
                                            .rename(columns={'UpdatedDate': 'Date', 'SpotPrice2': 'Silver 1kg'})
        silver_historical_df['Date'] = pd.to_datetime(silver_historical_df['Date'], format='%d-%b-%Y')
        silver_historical_df['Silver 1kg'] = silver_historical_df['Silver 1kg'].astype(float)

        # crudeoil_historical_data = nse_api._get_data(f"api/historical-spot-price?symbol=CRUDEOIL&fromDate={from_dt}&toDate={to_dt}")

        fii_dii_data_df = fetch_fii_dii_data(year=year)

        # market_status = get_nse_market_status_daily()

        return (nifty_50_historical_data, 
                india_vix_historical_data,
                gold_historical_df, 
                silver_historical_df,
                fii_dii_data_df)

In [7]:
nifty_50_historical_data, \
 india_vix_historical_data, \
 gold_historical_df, \
 silver_historical_df, \
 fii_dii_data_df = sourcing_data()

In [10]:
nifty50_historical_graph_df, nifty50_graph_identifier = load_graph_data_to_df(nifty_50_historical_data)

viz_df = pd.merge(nifty50_historical_graph_df.loc[:, ['Date', 'Price']].rename(columns={'Price': 'Nifty 50'}),
                  india_vix_historical_data[['date', 'close']].rename(columns={'date': 'Date', 'close': 'India VIX'}),
                  on='Date', how='left')
viz_df = pd.merge(viz_df, gold_historical_df, on='Date', how='left')
viz_df = pd.merge(viz_df, silver_historical_df, on='Date', how='left')
viz_df = pd.merge(viz_df, fii_dii_data_df, on='Date', how='left')

In [11]:
viz_df

,Date,Nifty 50,India VIX,Gold 10gm,Silver 1kg,FII Net,DII Net
0,2025-01-03,24004.75,13.5400,77210.0,87974.0,NaN,NaN
1,2025-01-06,23616.05,15.6500,76661.0,88117.0,NaN,NaN
2,2025-01-07,23707.90,14.6600,76859.0,89411.0,NaN,NaN
3,2025-01-08,23688.95,14.4650,77159.0,89567.0,NaN,NaN
4,2025-01-09,23526.50,14.6575,77381.0,89670.0,NaN,NaN
...,...,...,...,...,...,...,...
244,2025-12-29,25942.10,9.7200,136532.0,233131.0,-2759.89,2643.85
245,2025-12-30,25938.85,9.6800,134158.0,231542.0,-3844.02,6159.81
246,2025-12-31,26129.60,9.4800,NaN,NaN,-3597.38,6759.64
247,2026-01-01,26146.55,9.1900,NaN,NaN,NaN,NaN


In [12]:
# graph_df.loc[:, ['Date', 'Price']].rename(columns={'Price': 'Nifty 50'}).dtypes
# india_vix_1y_historical_data[['date', 'close']].rename(columns={'date': 'Date', 'close': 'India VIX'})

In [3]:
def get_nifty_data():
    # A mix of real symbols for realism
    symbols = [
        "COALINDIA", "NTPC", "HINDALCO", "SBIN", "JIOFIN", "BAJFINANCE", 
        "POWERGRID", "MARUTI", "BEL", "MAXHEALTH", "HINDUNILVR", "RELIANCE", 
        "M&M", "HDFCBANK", "ADANIENT", "INFY", "ASIANPAINT", "WIPRO", 
        "ADANIPORTS", "LT", "TCS", "ITC", "AXISBANK", "TITAN", "ULTRACEMCO"
    ]
    
    data = []
    for sym in symbols:
        price = np.random.uniform(200, 4000)
        # Generate varied percentage changes (-2% to +5%)
        # Biased towards positive to match your green screenshot
        change = np.random.uniform(-1.0, 5.0) 
        
        data.append({
            "Symbol": sym,
            "Price": round(price, 2),
            "Change": round(change, 2),
            "Size": 1  # Constant size to make all cards equal
        })
    
    return pd.DataFrame(data)

df = get_nifty_data()

In [4]:
df

,Symbol,Price,Change,Size
0,COALINDIA,3539.23,2.88,1
1,NTPC,2940.54,-0.85,1
2,HINDALCO,556.60,-0.38,1
3,SBIN,3587.53,4.02,1
4,JIOFIN,1793.85,-0.85,1
5,BAJFINANCE,3223.16,4.78,1
6,POWERGRID,1241.28,0.10,1
7,MARUTI,1462.72,2.97,1
8,BEL,913.36,2.48,1
9,MAXHEALTH,240.94,3.22,1


In [1]:
viz_df.iloc[:, ['Date', 'Nifty 50', 'India VIX']]

NameError: name 'viz_df' is not defined

In [29]:
latest_data

Date         2025-12-30 00:00:00
Nifty 50                25938.85
India VIX                   9.68
Gold 10gm                 134158
Name: 248, dtype: object

In [15]:
from time import sleep
from_dt = "30-12-2024"
to_dt = "30-12-2025"


nse_api = NSE_API()


gold_historical_data = nse_api._get_data(f"api/historical-spot-price?symbol=GOLD&fromDate={from_dt}&toDate={to_dt}")

sleep(2)

silver_historical_data = nse_api._get_data(f"api/historical-spot-price?symbol=SILVER&fromDate={from_dt}&toDate={to_dt}")

# https://www.nseindia.com/api/historical-spot-price?symbol=SILVER&fromDate=03-01-2025&toDate=03-01-2026
# https://www.nseindia.com/api/historical-spot-price?symbol=SILVER&fromDate=01-04-2024&toDate=31-12-2025


error from NSE_API.get_data(): 503 Server Error: Service Unavailable for url: https://www.nseindia.com/api/historical-spot-price?symbol=GOLD&fromDate=30-12-2024&toDate=30-12-2025
error from NSE_API.get_data(): 503 Server Error: Service Unavailable for url: https://www.nseindia.com/api/historical-spot-price?symbol=SILVER&fromDate=30-12-2024&toDate=30-12-2025


In [9]:
silver_historical_df = pd.DataFrame(silver_historical_data['data']) \
                                            .loc[:, ['UpdatedDate', 'SpotPrice2']] \
                                            .rename(columns={'UpdatedDate': 'Date', 'SpotPrice2': 'Silver 1kg'})
silver_historical_df['Date'] = pd.to_datetime(silver_historical_df['Date'], format='%d-%b-%Y')
silver_historical_df['Silver 1kg'] = silver_historical_df['Silver 1kg'].astype(float)

In [10]:
# silver_historical_data
silver_historical_df


,Date,Silver 1kg
0,2025-12-30,231542.0
1,2025-12-29,233131.0
2,2025-12-26,228695.0
3,2025-12-24,218918.0
4,2025-12-23,210554.0
...,...,...
244,2025-01-03,87974.0
245,2025-01-02,86977.0
246,2025-01-01,85996.0
247,2024-12-31,85977.0


In [14]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Data Simulation (Matching your provided start/end points) ---
def generate_market_data():
    dates = pd.date_range(start="2024-12-30", periods=250, freq='B') # Business days
    
    # Linear interpolation + Noise for realistic look
    nifty = np.linspace(23644, 25938, len(dates)) + np.random.normal(0, 150, len(dates))
    vix = np.linspace(13.97, 9.68, len(dates)) + np.random.normal(0, 0.5, len(dates))
    gold = np.linspace(75953, 134158, len(dates)) + np.random.normal(0, 500, len(dates))
    
    return pd.DataFrame({'Date': dates, 'Nifty 50': nifty, 'India VIX': vix, 'Gold': gold})

df = generate_market_data()

# --- 2. Streamlit Setup ---


# --- 3. Build Stacked Subplots ---
# rows=3, cols=1 creates a vertical stack
fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True, # vital for comparing timeframes
    vertical_spacing=0.05,
    subplot_titles=("Nifty 50 (Equity)", "India VIX (Volatility)", "Gold 10gm (Commodity)"),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}], [{"secondary_y": False}]]
)

# Row 1: Nifty 50
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Nifty 50'],
    name="Nifty 50", mode='lines',
    line=dict(color='#00E396', width=2) # Green
), row=1, col=1)

# Row 2: India VIX (Inverse indicator, often red/orange)
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['India VIX'],
    name="India VIX", mode='lines',
    line=dict(color='#FF4560', width=2) # Red
), row=2, col=1)

# Row 3: Gold (Safe haven, often yellow/gold)
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Gold'],
    name="Gold", mode='lines',
    line=dict(color='#FEB019', width=2) # Gold color
), row=3, col=1)

# --- 4. Professional Styling ---
fig.update_layout(
    height=800, # Taller chart to accommodate 3 panels
    template="plotly_dark",
    hovermode="x unified", # Shows all 3 values in one tooltip when hovering a date
    showlegend=False, # Hide legend to save space (titles are sufficient)
    margin=dict(t=50, b=50, l=50, r=50)
)

# Add Range Slider only to the bottom plot, but it controls all
fig.update_xaxes(matches='x') 


In [16]:
market_status = get_nse_market_status_daily()

In [18]:
market_status

,nifty50_close,nifty50_change,nifty50_perchange,nifty50_status,giftnifty_close,giftnifty_change,giftnifty_perchange,marketcapin_trdollars,marketcapin_laccr_rupees,as_of_date,current_date_time
0,25938.85,-3.22,-0.01,CLOSE,None,None,None,5.22,469.187764,30-Dec-2025 15:30,None


In [20]:

import pandas as pd

# 1. Setup Dummy Data (Matching your previous structure)
data = {
    "Date": ["2025-12-29", "2025-12-30"],
    "Nifty 50": [25942.10, 25938.85],  # Price dropped slightly
    "India VIX": [9.72, 9.68]          # Volatility dropped slightly
}
df = pd.DataFrame(data)

# 2. Calculation Logic
# Get the latest data (Last row)
latest_data = df.iloc[-1]
# Get the previous data (Second to last row)
prev_data = df.iloc[-2]

# Calculate Changes (Deltas)
nifty_change = latest_data["Nifty 50"] - prev_data["Nifty 50"]
vix_change = latest_data["India VIX"] - prev_data["India VIX"]

# 3. Create Columns
col1, col2 = st.columns(2)

# 4. Render Metrics
# f"{value:.2f}" formats the number to 2 decimal places

print(f"{latest_data['Nifty 50']:.2f}", f"{nifty_change:.2f}")

print(f"{latest_data['India VIX']:.2f}", f"{vix_change:.2f}")

2025-12-30 20:19:34.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:19:34.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:19:34.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


25938.85 -3.25
9.68 -0.04


In [22]:
df.iloc[-1]["Nifty 50"]

np.float64(25938.85)

In [ ]:
# Ray Dalio: bridgewater associates: Biggest Hedge Fund in the World
# warren buffett: berkshire hathaway: multinational holding company that owns entire businesses (like GEICO, Dairy Queen) and significant stakes in public companies
# charlie munger: berkshire hathaway vice chairman:

# 100 year old banner chart

# Inflation is ok?
# Interest Rate is stable or not?

In [ ]:
with st.sidebar:
    st.header("📲 Navigator")
    
    # Simulating a Navigation Menu using radio buttons
    page = st.radio(
        "Go to",
        ["Dashboard", "Market Analysis", "Portfolio Tracker", "TDF ChatBot", "Settings", "About Us"]
    )
    
    st.divider() # Adds a visual line separator

    # nifty_dashboard_page = st.selectbox(
    #     "Nifty Dashboard Pages",
    #     ["Nifty 50", "Nifty Bank"]
    # )
    
    st.info("Comming Soon: More features and pages!")
    st.divider()

    st.info("Guest User: Limited Access.")